<a href="https://colab.research.google.com/github/kavyajeetbora/CityHealthMonitor/blob/master/notebooks/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import geemap
import ee
from datetime import datetime, timedelta

ee.Authenticate()
ee.Initialize(project='kavyajeetbora-ee')

## Utility Functions

- Create a standard function to apply filters on image collection

In [28]:
def filterImageCollection(
        image_collection_id: str,
        geometry: ee.geometry.Geometry,
        date: tuple[str]
    ) -> ee.imagecollection.ImageCollection:

    imageCollection = ee.ImageCollection(image_collection_id)
    filtered = imageCollection.filter(
        ee.Filter.date(date[0], date[1]) ## Filter by date
    ).filter(
        ee.Filter.bounds(geometry) ## Filter by bounds
    )

    return imageCollection

In [29]:
admin = ee.FeatureCollection('FAO/GAUL_SIMPLIFIED_500m/2015/level2')
delhi = admin.filter(ee.Filter.eq('ADM1_NAME', 'Delhi'))\
.filter(ee.Filter.eq('ADM2_NAME', 'Delhi'))

geometry = delhi.geometry()

imageCollection = filterImageCollection(
    image_collection_id = 'COPERNICUS/S2_SR_HARMONIZED',
    geometry = geometry,
    date = ('2023-01-01', '2024-01-01')
)

In [31]:
s2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
admin = ee.FeatureCollection('FAO/GAUL_SIMPLIFIED_500m/2015/level2')

delhi = admin.filter(ee.Filter.eq('ADM1_NAME', 'Delhi'))\
.filter(ee.Filter.eq('ADM2_NAME', 'Delhi'))

geometry = delhi.geometry()

## Filtered_image_Collection
filtered_imgs = filterImageCollection(
    image_collection_id = 'COPERNICUS/S2_SR_HARMONIZED',
    geometry = geometry,
    date = ('2023-01-01', '2024-01-01')
)

image = filtered_imgs.median()
clipped_img = image.clip(geometry)

center = (28.6182,77.2009)
Map = geemap.Map(center=center, zoom=10)
Map.addLayer(ee_object=delhi, vis_params={'color':'red'}, name='Delhi Boundary')
viz_params = {
    'min': 0,
    'max': 3000,
    'bands': ['B4', "B3", "B2"]
}
Map.addLayer(clipped_img, viz_params, name='Sentinel')
Map

Map(center=[28.6182, 77.2009], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDa…

In [84]:
## get the current dates
today = datetime.now()
current_date_range = (f'01-01-{today.year-1}', f'01-01-{today.year}')
current_date_range

('01-01-2023', '01-01-2024')

In [97]:
## Filtered_image_Collection
surface_temp = filterImageCollection(
    image_collection_id = "MODIS/061/MOD11A1",
    geometry = geometry,
    date = current_date_range
).select('LST_Day_1km')

def kelvin_to_celsius(image):
    temp = image.subtract(273.15).rename('temp_C')
    return image.addBands(temp).copyProperties(image, ['system:time_start'])

In [107]:
temp_c = surface_temp.map(kelvin_to_celsius)
temp_c.size()

In [109]:
temp_mean = temp_c.median().clip(geometry)
temp_mean.bandNames()
print(temp_mean.getInfo())

{'type': 'Image', 'bands': [{'id': 'LST_Day_1km', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': 0, 'max': 65535}, 'dimensions': [2, 3], 'origin': [76, 27], 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'temp_C', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': -273.15, 'max': 65261.85}, 'dimensions': [2, 3], 'origin': [76, 27], 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}], 'properties': {'system:footprint': {'type': 'Polygon', 'coordinates': [[[76.83179521216374, 28.581733259417586], [76.83853247703775, 28.557029892152244], [76.85649854462561, 28.548046907845414], [76.85874427371164, 28.54130961869212], [76.8767103088653, 28.527835071946857], [76.8789561103673, 28.523343549963876], [76.87446455445522, 28.514360570339058], [76.8789561103673, 28.5098690290594], [76.88569335746607, 28.512114778560743], [76.89467636704018, 28.516606292563182], [76.89692210499737, 28.51885205825166], [76.90815088344674, 28.514360529769505], 

In [110]:
center = (28.6182,77.2009)
Map = geemap.Map(center=center, zoom=10)
viz_params = {
    'min': 0,
    'max': 50,
    'bands': ['LST_Day_1km']
}
Map.addLayer(temp_mean, {}, name='Surface Temperature')
Map

Map(center=[28.6182, 77.2009], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDa…

In [ ]:
# Load Landsat 8 image collection for surface temperature
landsat_collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA') \
    .filterBounds(geometry) \
    .filterDate('2019-01-01','2020-01-01') \
    .sort('CLOUD_COVER', True) \
    .select(['B10'])  # Selecting thermal infrared band (Band 10)

# Function to convert Landsat thermal infrared to surface temperature (in Celsius)
def kelvin_to_celsius(image):
    surface_temp_celsius = image.multiply(0.1).subtract(273.15).rename('surface_temperature_celsius')
    return image.addBands(surface_temp_celsius)

# Apply the conversion function to the image collection
landsat_collection_with_temp = landsat_collection.map(kelvin_to_celsius)

image = landsat_collection_with_temp.select('surface_temperature_celsius').median()
clipped_img = image.clip(geometry)

Map = geemap.Map(center=center, zoom=10)
vis_params = {
    'min': 10,  # Minimum surface temperature (Celsius)
    'max': 40,  # Maximum surface temperature (Celsius)
    'palette': ['blue', 'green', 'yellow', 'orange', 'red']  # Color palette for visualization
}

Map.addLayer(clipped_img, vis_params, name='Surface Temperature')
Map

Map(center=[28.6182, 77.2009], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDa…